In [5]:
!export LD_PRELOAD=/usr/lib/x86_64-linux-gnu/libstdc++.so.6

In [1]:
!pip install iree-base-runtime


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
# Download the model you want to compile and run.
!wget https://github.com/onnx/models/raw/refs/heads/main/validated/vision/classification/mobilenet/model/mobilenetv2-10.onnx

# Import to MLIR using IREE's ONNX importer.
!pip install iree-base-compiler[onnx]

--2025-02-07 21:20:32--  https://github.com/onnx/models/raw/refs/heads/main/validated/vision/classification/mobilenet/model/mobilenetv2-10.onnx
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/onnx/models/refs/heads/main/validated/vision/classification/mobilenet/model/mobilenetv2-10.onnx [following]
--2025-02-07 21:20:32--  https://media.githubusercontent.com/media/onnx/models/refs/heads/main/validated/vision/classification/mobilenet/model/mobilenetv2-10.onnx
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13963115 (13M) [application/octet-stream]
Saving to: ‘mobilenetv2-10.on

In [8]:
import os
os.environ['LD_PRELOAD'] = '/usr/lib/x86_64-linux-gnu/libstdc++.so.6'

!iree-import-onnx mobilenetv2-10.onnx --opset-version 17 -o mobilenetv2.mlir

In [11]:
!head -n 3 mobilenetv2.mlir

module {
  func.func @"torch-jit-export"(%arg0: !torch.vtensor<[?,3,224,224],f32>) -> !torch.vtensor<[?,1000],f32> attributes {torch.onnx_meta.ir_version = 6 : si64, torch.onnx_meta.opset_version = 17 : si64, torch.onnx_meta.producer_name = "pytorch", torch.onnx_meta.producer_version = "1.8"} {
    %0 = torch.operator "onnx.Constant"() {torch.onnx.value = dense_resource<_475> : tensor<32x3x3x3xf32>} : () -> !torch.vtensor<[32,3,3,3],f32> 


In [10]:
!iree-compile \
    --iree-hal-target-backends=cuda \
    --iree-cuda-target=sm_86 \
    mobilenetv2.mlir -o mobilenet_cpu.vmfb

In [14]:
!iree-run-module \
    --device=cuda \
    --module=mobilenet_cpu.vmfb \
    --function=torch-jit-export \
    --input="1x3x224x224xf32=0"

EXEC @torch-jit-export
result[0]: hal.buffer_view
1x1000xf32=[-4.54831 -8.24082 -2.19512 -0.607162 3.01811 2.2795 5.93878 -4.63611 -8.55423 -7.0313 -5.72422 -13.0843 -3.7323 -10.1492 -6.61326 -2.82114 -4.72417 -7.41983 -5.72615 -2.07717 -14.7411 -8.16984 -12.3777 -8.10809 -4.15393 -6.72145 -8.81415 -9.66889 -6.56354 -6.89167 -5.41167 -2.54415 1.90022 6.92701 6.78602 3.81411 4.48483 2.02717 -4.41525 -0.560704 -7.91978 -1.68703 -6.18004 -1.44641 -3.03691 0.652582 -6.68704 -7.32123 -7.23366 -8.32794 -2.4296 -7.09273 -0.798859 -3.02565 -3.67178 -3.66175 8.57692 -3.06189 -2.77678 -1.11859 -0.91623 4.8476 1.67301 3.86992 -7.21037 -1.28333 0.629251 -2.0707 -1.48169 -2.40855 -5.74169 2.41665 -3.04453 -2.30439 -5.12917 3.54654 6.30047 5.01499 1.57146 -5.38264 -13.8226 -4.57917 -5.10911 -4.07702 -13.281 3.7595 -4.19539 -3.61092 -0.711851 -1.47967 -8.88214 -1.92266 -3.45899 -1.75495 1.49693 3.88671 -2.84268 -7.44225 -2.31068 -5.51414 -10.6048 -11.8567 -6.41718 0.605824 -3.76109 0.966555 5.84405 4